In [ ]:
pip install azure-ai-documentintelligence==1.0.0b2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 6.9 MB/s eta 0:00:00


In [ ]:
# import libraries
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest

# set your endpoint and key variables
endpoint = "https://documentanalysiseswar.cognitiveservices.azure.com/"
key = "4b916a1b09ca451c985fa308044880ee"

# helper functions
def get_words(page, line):
    result = []
    for word in page.words:
        if _in_span(word, line.spans):
            result.append(word)
    return result

def _in_span(word, spans):
    for span in spans:
        if word.span.offset >= span.offset and (
            word.span.offset + word.span.length
        ) <= (span.offset + span.length):
            return True
    return False

def analyze_layout():
    # your specified document URL
    formUrl = "https://your.yale.edu/sites/default/files/ct-performance-feedback-completed-sample-form.pdf"

    document_intelligence_client = DocumentIntelligenceClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )

    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-read", AnalyzeDocumentRequest(url_source=formUrl)
    )

    result: AnalyzeResult = poller.result()

    if result.styles and any([style.is_handwritten for style in result.styles]):
        print("Document contains handwritten content")
    else:
        print("Document does not contain handwritten content")

    for page in result.pages:
        print(f"----Analyzing layout from page #{page.page_number}----")
        print(
            f"Page has width: {page.width} and height: {page.height}, measured with unit: {page.unit}"
        )

        if page.lines:
            for line_idx, line in enumerate(page.lines):
                words = get_words(page, line)
                print(
                    f"...Line # {line_idx} has word count {len(words)} and text '{line.content}' "
                    f"within bounding polygon '{line.polygon}'"
                )

                for word in words:
                    print(
                        f"......Word '{word.content}' has a confidence of {word.confidence}"
                    )

        if page.selection_marks:
            for selection_mark in page.selection_marks:
                print(
                    f"Selection mark is '{selection_mark.state}' within bounding polygon "
                    f"'{selection_mark.polygon}' and has a confidence of {selection_mark.confidence}"
                )

    if result.tables:
        for table_idx, table in enumerate(result.tables):
            print(
                f"Table # {table_idx} has {table.row_count} rows and "
                f"{table.column_count} columns"
            )
            if table.bounding_regions:
                for region in table.bounding_regions:
                    print(
                        f"Table # {table_idx} location on page: {region.page_number} is {region.polygon}"
                    )
            for cell in table.cells:
                print(
                    f"...Cell[{cell.row_index}][{cell.column_index}] has text '{cell.content}'"
                )
                if cell.bounding_regions:
                    for region in cell.bounding_regions:
                        print(
                            f"...content on page {region.page_number} is within bounding polygon '{region.polygon}'"
                        )

    print("----------------------------------------")

if __name__ == "__main__":
    analyze_layout()


Document does not contain handwritten content
----Analyzing layout from page #1----
Page has width: 11 and height: 8.5, measured with unit: inch
...Line # 0 has word count 1 and text 'Yale' within bounding polygon '[0.4392, 0.3438, 1.375, 0.3772, 1.375, 0.7497, 0.4392, 0.7545]'
......Word 'Yale' has a confidence of 0.993
...Line # 1 has word count 3 and text 'SAMPLE COMPLETED FORM' within bounding polygon '[3.4996, 0.4202, 7.0517, 0.4202, 7.0517, 0.616, 3.4996, 0.616]'
......Word 'SAMPLE' has a confidence of 0.996
......Word 'COMPLETED' has a confidence of 0.996
......Word 'FORM' has a confidence of 0.993
...Line # 2 has word count 5 and text 'CLERICAL AND TECHNICAL PERFORMANCE FEEDBACK' within bounding polygon '[0.4918, 0.936, 7.3477, 0.9407, 7.3477, 1.1556, 0.4918, 1.1508]'
......Word 'CLERICAL' has a confidence of 0.995
......Word 'AND' has a confidence of 0.999
......Word 'TECHNICAL' has a confidence of 0.994
......Word 'PERFORMANCE' has a confidence of 0.994
......Word 'FEEDBACK' 